### decompose

In [2]:
from prompt_manager import get_k_shot_with_answer, view_instruction, row_instruction
import pandas as pd
from utils import parse_specific_composition
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI, OpenAI
from data_loader import TableFormat, TableLoader
from langchain.memory import ChatMessageHistory
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from sqlalchemy import create_engine
from executor import SQLManager
import sqlparse

In [4]:
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from data_loader import TableFormat
query_examples = [
    # "after 2005 , the winner of the lifetime achievement award be andrew rule john silvester , sandra harvey lindsay simpson , marele day , shane maloney , and peter doyle",
                  "all 12 club play a total of 22 game for the wru division one east",
                #   "a gamecube game loss the award in each of the first 3 year",
                "from 1980 to 2011 , apoel bc lose more than 2 time as many game as it win",
                  "polona hercog 1890partner with alberta brianti after she have stephanie vogt as the partner",
                  ]
task_examples = ["query rewrite", "query decompose", "query ambiguity resolve"]
new_query_examples = [
    # "Who were the winners of the lifetime achievement award after 2005?;",
                      "How many clubs play for the wru division one east in total?; How many clubs play 22 game for the wru division one east?;",
                    #   "a gamecube game loss the award in each of the first 3 year",
                    "from 1980 to 2011 , how many games did apoel bc lose?; from 1980 to 2011 , how many games did apoel bc win?;",
                      "When did polona hercog partner with alberta brianti?; When did polona hercog partner with stephanie vogt?",
                      ]
num_k = 3
inds = [1, 124, 5]
table_loader = TableLoader(table_name='tabfact', split='validation', use_sample=True, small_test=False)
examples = [TableFormat(format='none', data=table_loader.dataset[inds[i]], use_sampling=True).format_html(table_loader.dataset[inds[i]]['table']['caption']) for i in range(num_k)]

examples_prompt = PromptTemplate(input_variables=["query", "table", "new_query"], template=
"""Query: {query}
Sub-Table: {table}
new_query: {new_query}""")

examples_dict = [{"query": query_examples[i],
                                    "table": examples[i],
                                    "new_query": new_query_examples[i]} for i in range(num_k)]
decompose_prompt = FewShotPromptTemplate(
    examples=examples_dict,
    example_prompt=examples_prompt,
    prefix="""You are capable of converting complex query into sub-queries. Based on the table, provide at most 2 continuity sub-queries for knowledge that you need. 
Split the queries with ’;’.""",
    suffix=
    """Query: {query}
Sub-Table: {table}
new_query: 
    """,
    input_variables=["query", "table"],
)

# Sub-questions are separated by semicolons.
# answer_instruction = PromptTemplate(input_variables=["SQL", "table", "claim"], 
#                                     template="""
# Below is a sub-table generated by excuting the SQL. You need to understand the logic behind the SQL filtering and complete task using the final sub-table. 
# SQL Excuted: 
# ```{SQL}```
# Sub-table: {table}
# Query: {claim}
# answer the question given in the query. Only return the string instead of other format information. Do not repeat the question.
# """ )


In [5]:
task_name = 'tabfact'
split = 'test'
model_name = 'gpt-3.5-turbo-0125'
model = ChatOpenAI(model_name=model_name, openai_api_base="https://api.chatanywhere.tech/v1",
                       openai_api_key="sk-kxgtm71G6zwC44lglIF5CfiEVVzjjc39TOtppkNAwrVA2fUW", temperature=0.1)
schema_information = pd.read_csv(f"result/aug/{task_name}_{split}_schema.csv", index_col='table_id')
aug_information = pd.read_csv(f"result/aug/{task_name}_{split}_summary.csv", index_col='table_id')
composition_information = pd.read_csv(f"result/aug/{task_name}_{split}_composition.csv", index_col='table_id')
engine = create_engine('sqlite:///db/sqlite/tabfact.db', echo=False)
manager = SQLManager(engine=engine)

In [34]:
table_loader = TableLoader(table_name='tabfact', split='test', use_sample=False, small_test=True)
sample = table_loader.normalize_table(table_loader.dataset[2])

### step-back

In [9]:
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from data_loader import TableFormat
query_examples = [
    # "the country of ecuador be the only country that use the us dollar (usd) as its currency , and the central bank be the federal reserve",
                  "only 1 of the college list be public , and it be in new orleans"]
task_examples = ["query rewrite", "query decompose", "query ambiguity resolve"]
inds = [8]
num_k = 1
table_loader = TableLoader(table_name='tabfact', split='validation', use_sample=True, small_test=False)
examples = [TableFormat(format='none', data=table_loader.dataset[inds[i]], use_sampling=True).format_html(table_loader.dataset[inds[i]]['table']['caption']) for i in range(num_k)]
new_query_examples = [
    # "Which country uses the US dollar as its currency and has the Federal Reserve as its central bank?",
    "which college list be public?",
                    #   "what is the number of listings from barrington?; what is the number of listings from farmington?; what is the number of listings from rochester combined?",
                      ]
examples_prompt = PromptTemplate(input_variables=["query", "new_query"], template=
"""
Query: {query}
Sub-Table: {table}
new_query: {new_query}""")

examples_dict = [{"query": query_examples[i],
                  "table": examples[i],
                    "new_query": new_query_examples[i]} for i in range(num_k)]
step_back_prompt = FewShotPromptTemplate(
    examples=examples_dict,
    example_prompt=examples_prompt,
    prefix="""Based on the table, your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer.""",
    suffix=
    """
Query: {query}
Sub-Table: {table}
    """,
    input_variables=["query", "table"],
)


In [17]:

table_loader = TableLoader(table_name='tabfact', split='test', use_sample=False, small_test=True)
i = 31
sample = table_loader.normalize_table(
                    table_loader.dataset[i])

all_queries = []
formatter = TableFormat(format='none', data=sample, use_sampling=True)
llm_chain = LLMChain(llm=model, prompt=decompose_prompt, verbose=True)
batch_pred = llm_chain.batch([{"query": sample['query'], "table": formatter.format_html()}], return_only_outputs=True)



> Entering new LLMChain chain...
Prompt after formatting:
You are capable of converting complex query into sub-queries. Based on the table, provide at most 2 continuity sub-queries for knowledge that you need. 
Split the queries with ’;’.

Query: all 12 club play a total of 22 game for the wru division one east
Sub-Table: <table>
<caption>wru division one east</caption>
<thead>
<tr><th>            club</th><th>  played</th><th>  drawn</th><th>  lost</th><th>  points_for</th><th>  points_against</th><th>  tries_for</th><th>  tries_against</th><th>  try_bonus</th><th>  losing_bonus</th><th>  points</th></tr>
</thead>
<tbody>
<tr><td>fleur de lys rfc</td><td>22      </td><td>1      </td><td>16    </td><td>300         </td><td>617             </td><td>34         </td><td>77             </td><td>2          </td><td>4             </td><td>28      </td></tr>
<tr><td>beddau rfc      </td><td>22      </td><td>0      </td><td>15    </td><td>310         </td><td>483             </td><td>32     

In [18]:
batch_pred[0]['text']

'How many players are from the United States?; How many players are from Canada?'

In [10]:
def get_k_shot_with_answer(k: int=1):
    sqls = ["SELECT MIN(points) FROM DF WHERE rider = 'roger dutton / tony wright';"]
    thoughts = ["Based on the SQL query provided, the minimum number of points that Roger Dutton / Tony Wright received in the 1972 Isle of Man TT event was 3. Therefore, the claim that 2 is the fewest points they received is false. The output should be 0."]
    tables = ["<table>\n<caption>1972 isle of man tt</caption>\n<thead>\n<tr><th>  MIN(points)</th></tr>\n</thead>\n<tbody>\n<tr><td>3            </td></tr>\n</tbody>\n</table>"]
    claims = ["2 be the fewest point that roger dutton / tony wright receive"]
    # inds from test split
    examples_prompt = PromptTemplate(input_variables=["SQL", "table", "claim", "thought", "output"], template=
    """
    SQL Excuted: 
    ```{SQL}```
    Sub-table: {table}
    ...

    Query: {claim}
    Thought: {thought}
    Output: {output}
    """)
    examples_dict = dict(zip(["SQL", "table", "claim", "thought", "output"], [sqls[0], tables[0], claims[0], thoughts[0], '0']))
    prompt_template = FewShotPromptTemplate(
        examples=[examples_dict],
        example_prompt=examples_prompt,
        prefix="""Below are some sub-tables generated by excuting the SQL. You need to understand the logic behind the SQL filtering. Complete task using all information below. Please think step by step and return 0 or 1 without any other information at last.""",
        suffix=
        """
    {information}
    Query: {query}
    Thought: """,
        input_variables=["table", "query"],
)
    return prompt_template

In [11]:
def get_k_shot_with_aug(k: int=2):
    table_loader = TableLoader(table_name='tabfact', split='validation', use_sample=True, small_test=False)

    inds = [3, 6]
    Output_examples = [
                       'team, goals_for',
                       'year, game, platform_s']
    linking_examples = ['the team -> team; the most goal for -> goals_for',
                        'gamecube -> platform_s; gamecube game -> game; the first 3 year -> year;'
    ]
    examples_prompt = PromptTemplate(input_variables=["table", "claim", "output", "linking"], template=
    """
    Table: {table}
    Claim: {claim}
    Column linking: {linking}
    Columns: {output}""")
    num_k = 2
    examples_dict = [{"table": TableFormat(format='none', data=table_loader.dataset[inds[i]], use_sampling=True).format_html(table_loader.dataset[inds[i]]['table']['caption']),
                                        "claim": table_loader.dataset[inds[i]]['statement'],
                                        "linking": linking_examples[i],
                                        # "summary": summary_examples[i],
                                        "output": Output_examples[i]} for i in range(num_k)]
    prompt_template = FewShotPromptTemplate(
        examples=examples_dict,
        example_prompt=examples_prompt,
        prefix=
        """
    Your task is accurately output columns related to the query or contain useful information about the query. This process involves linking relevant columns in the table on the terms in the query.
    Approach this task as follows:
    Read the question thoroughly and list every possible link from query term to column in Table.
    Based on the linking columns, output all useful columns at last. Make sure all columns in the link step are included and every column is in the Table.
    """,
    # You are a brilliant table executor with the capabilities information retrieval, table parsing, table partition and semantic understanding who can understand the structural information of the table.
    # Given the following table and query, you should output columns related to the query or contain useful information about the query. 
    # Here are some examples:""",
        suffix=
        """
    Table: {table}
    Claim: {claim}
    Extra information: {aug}
    """,
        input_variables=["table", "claim", "aug"],
)
    return prompt_template

In [ ]:
sample = table_loader.normalize_table(table_loader.dataset[0])
k_shot_prompt = get_k_shot_with_aug()
formatter = TableFormat(format='none', data=sample, use_sampling=True)
llm_chain = LLMChain(llm=model, prompt=k_shot_prompt, verbose=True)
summary_aug, column_aug = aug_information.loc[sample['id']]['summary'], aug_information.loc[sample['id']]['column_description'] 
extra_information = '\n'.join(parse_specific_composition(composition_information.loc[sample['id']]['composition'], formatter.data.columns))
stage_1_batch_pred = llm_chain.batch([dict({'table': formatter.format_html(table_caption=sample['table']['caption']),
                                        #     'claim': 'Who is one of the three nominees for a Drama Desk Award?',
                                    'claim': sample['query'],
                                    'aug':  extra_information
                                    })], return_only_outputs=True)[0]['text']
# pred = llm_chain.batch([dict({"query": 'Who is one of the three nominees for a Drama Desk Award?'})])
# print(pred[0]['text'])
print(stage_1_batch_pred)


In [19]:
from utils import parse_output
def scene_A(query, sample, verbose=True):
    row_instruction = PromptTemplate(input_variables=["table", "claim", "aug"], 
                                 template="""
Our ultimate goal is to answer query based on the table. Below is a subtable with columns filtered, you are required to infer the data distribution and format from the sample data of the sub-table. Carefully analyze the query, based on the augmentation information, write a SQLITE3 SELECT SQL statement using table DF that complete query. Directly Output SQL, do not add other string.
sub-table: {table}
Query: {claim}
Extra information: {aug}
SQL: """)
    formatter = TableFormat(format='none', data=sample, use_sampling=True)
    k_shot_prompt = get_k_shot_with_aug()
    
    with get_openai_callback() as cb:
        llm_chain = LLMChain(llm=model, prompt=k_shot_prompt, verbose=verbose)
        summary_aug, column_aug = aug_information.loc[sample['id']]['summary'], aug_information.loc[sample['id']]['column_description'] 
        col_names, col_infos = parse_output(column_aug, pattern=r'([^<]*)<([^>]*)>')
        extra_col_info = []
        for i_c in range(len(col_names)):
            extra_col_info.append(f'{i_c + 1}. {col_names[i_c]}: {col_infos[i_c]}')
        stage_1_batch_pred = llm_chain.batch([dict({'table': formatter.format_html(table_caption=sample['table']['caption']),
                                            'claim': query,
                                            'aug':  summary_aug + '\n'.join(extra_col_info)
                                            })], return_only_outputs=True)[0]['text']
        print(stage_1_batch_pred)
        stage_1_batch_pred = stage_1_batch_pred.split(':')[-1]
        
        # stage 2: SQL generation
        
        llm_chain = LLMChain(llm=model, prompt=row_instruction, verbose=verbose)
        columns = [formatter.normalize_col_name(c.strip()) for c in stage_1_batch_pred.split(',')]
        formatter.normalize_schema(schema_information.loc[sample['id']]['schema'])
        try: 
            formatter.data = formatter.data.loc[:, columns]
            formatter.all_data = formatter.all_data.loc[:, columns]
        except:
            pass
        extra_information = '\n'.join(parse_specific_composition(composition_information.loc[sample['id']]['composition'], formatter.data.columns))
        stage_2_batch_pred = llm_chain.batch([dict({'table': formatter.format_html(table_caption=sample['table']['caption']),
                                            'claim': query,
                                            'aug':  summary_aug + '\n Column information:' + extra_information
                                            })], return_only_outputs=True)[0]['text']
    print("total_tokens:", cb.total_tokens)
    print(stage_2_batch_pred)
    # stage 3: SQL Excution
    try: 
        formatter.data = manager.execute_from_df(stage_2_batch_pred, formatter.all_data, table_name='DF')
    except:
        formatter.data = formatter.all_data
        stage_2_batch_pred = 'SELECT * from DF;'
    if len(formatter.data) == 0:
        return query, stage_2_batch_pred, 'No data from database', cb.total_tokens
    return query, stage_2_batch_pred, formatter.format_html(), cb.total_tokens

In [12]:
inds = [0,   1,  11,  14,  22,  24,  26,  31,  37,  38,  39,  40,  47,
        49,  50,  59,  62,  63,  65,  66,  68,  69,  70,  75,  80,  81,
        82,  90,  91,  93,  95,  99, 100, 105, 106, 108, 110, 116, 119,
       124, 129, 136, 137, 145, 147, 148, 151, 153, 155, 158, 160, 164,
       167, 169, 171, 173, 175, 179, 180, 181, 187, 188, 190, 192, 198,
       200, 207, 209, 210, 211, 217, 225, 227, 230, 233, 236, 237, 241,
       251]

In [13]:
import concurrent.futures
from langchain_community.callbacks import get_openai_callback
def parallel_run(func, args_list):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = [executor.submit(func, arg) for arg in args_list]
        return [future.result() for future in concurrent.futures.as_completed(results)]

def parallel_run_kwargs(func, args_list):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = executor.map(lambda kwargs: func(**kwargs), args_list)
        return list(results)

In [11]:
from typing import List
import os
import json
def save_csv(input_list: List[List], label_list: List, file_path):
    import pandas as pd
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)

    assert len(input_list) == len(label_list)
    df = pd.DataFrame()
    for i in range(len(label_list)):
        df[label_list[i]] = pd.Series(input_list[i])
    if os.path.exists(file_path) and file_path.endswith('.csv'):
        df_origin = pd.read_csv(file_path)
        df = pd.concat([df_origin, df], axis=0)
    df.to_csv(file_path, index=False, encoding='utf-8')

### RUN

In [67]:
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI, OpenAI
import datetime
model = ChatOpenAI(model_name='gpt-3.5-turbo-0125', openai_api_base="https://api.chatanywhere.com.cn/v1",
                       openai_api_key="sk-kxgtm71G6zwC44lglIF5CfiEVVzjjc39TOtppkNAwrVA2fUW", temperature=0.1)
table_loader = TableLoader(table_name='tabfact', split='test', use_sample=False, small_test=True)

save_path = f"result/answer/tabfact_zh_{datetime.datetime.now().strftime('%m-%d_%H-%M-%S')}.csv"
muilti_answer_instruction = PromptTemplate(input_variables=["information", "claim"], 
template="""You are a brilliant table executor with the capabilities information retrieval, table parsing, table partition and semantic understanding who can understand the structural information of the table.
Below are some sub-tables, each sub-table is generated by excuting the corresponding SQL. You need to understand the logic behind the SQL filtering. Complete task by using useful information below.
You can use one of the sub-tables or use all sub-tables.
{information}
Query: {query}
verify whether the provided claim/query is true or false, return 0 if it's false, or 1 if it's true. Please think step by step and return 0/1 at last.
""" )
tokens = []
outputs = []
labels = []
stage_1 = []
stage_2 = []
# muilti_answer_instruction = get_k_shot_with_answer()
i = 
    
sample = table_loader.normalize_table(
                    table_loader.dataset[i])
labels.append(sample['label'])
all_queries = [sample['query']]
formatter = TableFormat(format='none', data=sample, use_sampling=True)
llm_chain = LLMChain(llm=model, prompt=step_back_prompt, verbose=False)
batch_pred = llm_chain.batch([{"query": sample['query']}], return_only_outputs=True)
all_queries.append(batch_pred[0]['text'].split(':')[-1])
print(batch_pred[0]['text'].split(':')[-1])

args_list = [{"query": q, "sample": sample} for q in all_queries]
results = parallel_run_kwargs(scene_A, args_list)
temp = [f"""
SQL Excuted: 
```{res[1]}```
Sub-table: {res[2]}""" for res in results if len(res[1]) > 0]
tokens.append(sum([res[3] for res in results]) / len(results))
llm_chain = LLMChain(llm=model, prompt=muilti_answer_instruction, verbose=True)
batch_pred = llm_chain.batch([{"query": sample['query'], "information": '\n'.join(temp)}], return_only_outputs=True)
print(batch_pred[0])
outputs.append(batch_pred[0]['text'])
# llm_chain = LLMChain(llm=model, prompt=decompose_prompt, verbose=False)
# batch_pred = llm_chain.batch([{"query": sample['query'], "table": formatter.format_html(table_caption=sample['table']['caption'])}], return_only_outputs=True)
# print(batch_pred[0]['text'].split(':')[-1].split(';'))
# all_queries.extend(batch_pred[0]['text'].split(':')[-1].split(';'))
# "Is the following query true or false?" +

SyntaxError: invalid syntax (1272733091.py, line 23)

In [29]:
acc = 0
non_equal = []
for ind, (pred, gold) in enumerate(zip(table['pred'], table['label'] )):
            if pred == str(gold):
                acc += 1
            else:
                non_equal.append(ind)

In [57]:
non_equal

[1,
 3,
 11,
 14,
 22,
 24,
 30,
 32,
 33,
 34,
 37,
 38,
 39,
 40,
 41,
 44,
 47,
 51,
 52,
 58,
 59,
 63,
 64,
 65,
 66,
 69,
 76,
 80,
 83,
 88]

In [63]:
print(table.iloc[75, :]['output'])

To verify the claim that "pierre lamine beat shinji someya by more than a point", we need to compare the points of both individuals from the sub-table provided.

From the sub-table:
- pierre lamine has 150.5 points
- shinji someya has 150.34 points

To determine if pierre lamine beat shinji someya by more than a point, we need to calculate the point difference between them:
150.5 - 150.34 = 0.16

Since the point difference is less than 1, the claim that pierre lamine beat shinji someya by more than a point is false.

Therefore, the answer is 0.


In [29]:
non_equals = []
for i in non_equal:
    non_equals.append(inds[i])

In [9]:
import pandas as pd
table = pd.read_csv('./result/answer/tabfact_zh_04-23_15-22-00.csv', encoding='utf-8')
table.head()
table['pred'] = eval_blury_string(table['output'])

In [33]:
def eval_blury_string(pred_list):
    pred_label = []
    for pred in pred_list:
        predict_ans = pred.split('\n')[-1]
        if '0' in predict_ans:
            predict_ans = '0'
        elif '1' in predict_ans:
            predict_ans = '1'
        else:
            predict_ans = '2'
        pred_label.append(predict_ans)
    return pred_label

In [5]:
import concurrent.futures
from langchain_community.callbacks import get_openai_callback
def parallel_run(func, args_list):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = [executor.submit(func, arg) for arg in args_list]
        return [future.result() for future in concurrent.futures.as_completed(results)]

def parallel_run_kwargs(func, args_list):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = executor.map(lambda kwargs: func(**kwargs), args_list)
        return list(results)
# args_list = [{"query": 'Who took the loss in the game on August 30 and who suffered the loss in the game on August 31?', "sample": sample},{"query": sample['query'], "sample": sample}]
# args_list = [{"query": q, "sample": sample} for q in all_queries]

# results = parallel_run_kwargs(scene_A, args_list)
# print(results)
# print(cb.total_tokens)


In [ ]:
muilti_answer_instruction = PromptTemplate(input_variables=["information", "claim"], 
                                    template="""
Below are some sub-tables generated by excuting the SQL. You need to understand the logic behind the SQL filtering. Complete task using all information below. 
{information}
Query: {query}
verify whether the provided claim/query is true or false, return 0 if it's false, or 1 if it's true. Please think step by step.
""" )
temp = [f"""
SQL Excuted: 
```{res[1]}```
Sub-table: {res[2]}""" for res in results]
muilti_answer_instruction = get_k_shot_with_answer()
llm_chain = LLMChain(llm=model, prompt=muilti_answer_instruction, verbose=True)
batch_pred = llm_chain.batch([{"query": sample['query'], "information": '\n'.join(temp)}], return_only_outputs=True)

In [35]:
def scene_B():
    agent_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """Based on the history information, your task is to only based on the conversation information to answer the user query.
    If you cannot get the answer from past history, reorganize the question and return the question explicitly. If you are confident in the answer, answer it directly.""",
        ),
        MessagesPlaceholder(variable_name="chat_history"),
    ]
)
    chain = LLMChain(llm=model, prompt=agent_prompt, verbose=True)
    return chain.invoke(
    {
        "chat_history": Agent_history.messages,
    }
)['text']
    
    #维护一个Agent Memory
Agent_history = []
agent_prompt = ChatPromptTemplate.from_messages(
    [
        # (
        #     "system",
        #     """"""
            
        #     # return 'A' if you think you need more information, else return 'B'. You should only use information during the conversation.,
        # ),
        MessagesPlaceholder(variable_name="chat_history"),
    ]
)
system =  PromptTemplate(input_variables=["history"], template="""Based on the history information, your task is to only based on the conversation information to answer the user query. 
Note: Do not use information on your own, only use information from the conversation history!

conversation histroy:
{history}

The output should choose from Choice A and Choice B:
Choice A: ###If you cannot get the answer from conversation histroy, reorganize the question and return the question explicitly.
Choice B: ###If you are confident in the answer, answer it directly.""")
chain = LLMChain(llm=model, prompt=system, verbose=True)
Agent_history.append('Q: Who were the winners of the lifetime achievement award after 2005?')
# Agent_history.append('A: the winners are andrew rule john silvester , sandra harvey lindsay simpson , marele day , shane maloney , and peter doyle')
# Agent_history.append('Q: Are the winners andrew rule john silvester , sandra harvey lindsay simpson , marele day , shane maloney , and peter doyle?')


In [ ]:
import concurrent.futures
 
def outer_task(url):
   # 外层任务
   print(f"Processing {url}")
   # 内部再次使用线程池
   with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
       # 执行一些依赖于外层任务的结果的并行操作
       inner_results = [executor.submit(inner_task, f"{url}_{i}") for i in range(2)]
       # 等待内部任务完成并收集结果
       return [r.result() for r in inner_results]
 
def inner_task(url):
   # 内层任务
   print(f"Inner task for {url}")
   # 这里可以执行一些操作，比如I/O密集型的任务
   return f"Result for {url}"
 
if __name__ == "__main__":
   # 创建线程池
   with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
       # 提交外层任务到线程池
       outer_results = [executor.submit(outer_task, f"http://example.com/{i}") for i in range(2)]
       # 等待外层任务完成并收集结果
       for future in concurrent.futures.as_completed(outer_results):
           print(future.result())

## 调整extrainformation的位置

In [41]:
from utils import parse_output
answer_instruction = PromptTemplate(input_variables=["SQL", "table", "claim"], 
                                    template="""
Below is a sub-table generated by excuting the SQL. You need to understand the logic behind the SQL filtering and complete task using the final sub-table. 
SQL Excuted: 
```{SQL}```
Sub-table: {table}
Query: {claim}
answer the question given in the query. If you cannot answer the question based on the sub-table, just say 'Cannot get answer from sub-table'
""" )
def scene_B(query, sample, verbose=True):
    row_instruction = PromptTemplate(input_variables=["table", "claim", "aug"], 
                                 template="""
Our ultimate goal is to answer query based on the table. Below is a subtable with columns filtered, you are required to infer the data distribution and format from the sample data of the sub-table. Carefully analyze the query, based on the augmentation information, write a SQLITE3 SELECT SQL statement using table DF that complete query. Directly Output SQL, do not add other string.
sub-table: {table}
Query: {claim}
Extra information: {aug}
SQL: """)
    formatter = TableFormat(format='none', data=sample, use_sampling=True)
    k_shot_prompt = get_k_shot_with_aug()
    
    with get_openai_callback() as cb:
        llm_chain = LLMChain(llm=model, prompt=k_shot_prompt, verbose=verbose)
        summary_aug, column_aug = aug_information.loc[sample['id']]['summary'], aug_information.loc[sample['id']]['column_description'] 
        col_names, col_infos = parse_output(column_aug, pattern=r'([^<]*)<([^>]*)>')
        extra_col_info = []
        for i_c in range(len(col_names)):
            extra_col_info.append(f'{i_c + 1}. {col_names[i_c]}: {col_infos[i_c]}')
        stage_1_batch_pred = llm_chain.batch([dict({'table': formatter.format_html(table_caption=sample['table']['caption']),
                                            'claim': query,
                                            'aug':  summary_aug + '\n'.join(extra_col_info)
                                            })], return_only_outputs=True)[0]['text']
        print(stage_1_batch_pred)
        stage_1_batch_pred = stage_1_batch_pred.split(':')[-1]
        
        # stage 2: SQL generation
        
        llm_chain = LLMChain(llm=model, prompt=row_instruction, verbose=verbose)
        columns = [formatter.normalize_col_name(c.strip()) for c in stage_1_batch_pred.split(',')]
        formatter.normalize_schema(schema_information.loc[sample['id']]['schema'])
        try: 
            formatter.data = formatter.data.loc[:, columns]
            formatter.all_data = formatter.all_data.loc[:, columns]
        except:
            pass
        extra_information = '\n'.join(parse_specific_composition(composition_information.loc[sample['id']]['composition'], formatter.data.columns))
        stage_2_batch_pred = llm_chain.batch([dict({'table': formatter.format_html(table_caption=sample['table']['caption']),
                                            'claim': query,
                                            'aug':  summary_aug + '\n Column information:' + extra_information
                                            })], return_only_outputs=True)[0]['text']
    
        print(stage_2_batch_pred)
        # stage 3: SQL Excution
        try: 
            formatter.data = manager.execute_from_df(stage_2_batch_pred, formatter.all_data, table_name='DF')
        except:
            formatter.data = formatter.all_data
            stage_2_batch_pred = 'SELECT * from DF;'
        
        llm_chain = LLMChain(llm=model, prompt=answer_instruction, verbose=verbose)
        response = llm_chain.batch([dict({'table': formatter.format_html(),
                                                'claim': query,
                                                'SQL':  stage_2_batch_pred
                                                })], return_only_outputs=True)[0]['text']
    print("total_tokens:", cb.total_tokens)
    return response, cb.total_tokens

In [46]:
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI, OpenAI
import datetime
model = ChatOpenAI(model_name='gpt-3.5-turbo-0125', openai_api_base="https://api.chatanywhere.tech/v1",
                       openai_api_key="sk-kxgtm71G6zwC44lglIF5CfiEVVzjjc39TOtppkNAwrVA2fUW", temperature=0.1)
table_loader = TableLoader(table_name='tabfact', split='test', use_sample=False, small_test=True)

save_path = f"result/answer/tabfact_zh_{datetime.datetime.now().strftime('%m-%d_%H-%M-%S')}.csv"
muilti_answer_instruction = PromptTemplate(input_variables=["information", "claim"], 
template="""You are a brilliant table executor with the capabilities information retrieval, table parsing, table partition and semantic understanding who can understand the structural information of the table.
Below is a sub-table generated by excuting the corresponding SQL. You need to understand the logic behind the SQL filtering. Complete task with the help of extra information below.
SQL Excuted: 
```{SQL}```
Sub-table: {table}
Extra information: {information}
Query: {query}
verify whether the provided claim/query is true or false, return 0 if it's false, or 1 if it's true. Please think step by step and return 0/1 at last.
""" )
tokens = []
outputs = []
labels = []
# muilti_answer_instruction = get_k_shot_with_answer()
# for i in range(30, 41):
i = 70
sample = table_loader.normalize_table(
                    table_loader.dataset[i])
labels.append(sample['label'])
all_queries = []
formatter = TableFormat(format='none', data=sample, use_sampling=True)
llm_chain = LLMChain(llm=model, prompt=step_back_prompt, verbose=False)
batch_pred = llm_chain.batch([{"query": sample['query'], "table": formatter.format_html()}], return_only_outputs=True)
all_queries.append(batch_pred[0]['text'].split(':')[-1])
llm_chain = LLMChain(llm=model, prompt=decompose_prompt, verbose=False)
batch_pred = llm_chain.batch([{"query": sample['query'], "table": formatter.format_html()}], return_only_outputs=True)
all_queries.extend(batch_pred[0]['text'].split(';'))
print(len(all_queries))
args_list = [{"query": q, "sample": sample} for q in all_queries]
ans_from_B = parallel_run_kwargs(scene_B, args_list)
results = [res[0] for res in ans_from_B if res != 'Cannot get answer from sub-table']
all_tokens = sum([res[1] for res in ans_from_B])
#With answer
with get_openai_callback() as cb:
    imp_input = scene_A(sample['query'], sample, False)
    llm_chain = LLMChain(llm=model, prompt=muilti_answer_instruction, verbose=True)
    batch_pred = llm_chain.batch([{"query": sample['query'],"SQL": imp_input[1], "table": imp_input[2], "information": '\n'.join(results)}], return_only_outputs=True)
print(batch_pred[0])
all_tokens += cb.total_tokens
print('ALL TOKENS:', all_tokens)
outputs.append(batch_pred[0]['text'])

#With no answer
# temp = [f"""
# SQL Excuted for extra information: 
# ```{res[1]}```
# Sub-table for extra information: {res[2]}""" for res in results if res[2] != 'No data from database']
# imp_input = scene_A(sample['query'], sample, False)
# llm_chain = LLMChain(llm=model, prompt=muilti_answer_instruction, verbose=True)
# batch_pred = llm_chain.batch([{"query": sample['query'],"SQL": imp_input[1], "table": imp_input[2], "information": '\n'.join(temp)}], return_only_outputs=True)
# print(batch_pred[0])
# outputs.append(batch_pred[0]['text'])


3


> Entering new LLMChain chain...
Prompt after formatting:

    Your task is accurately output columns related to the query or contain useful information about the query. This process involves linking relevant columns in the table on the terms in the query.
    Approach this task as follows:
    Read the question thoroughly and list every possible link from query term to column in Table.
    Based on the linking columns, output all useful columns at last. Make sure all columns in the link step are included and every column is in the Table.
    


    Table: <table>
<caption>1986 - 87 north west counties football league</caption>
<thead>
<tr><th>  position</th><th>               team</th><th>  played</th><th>  drawn</th><th>  lost</th><th>  goals_for</th><th>  goals_against</th><th>  goal_difference</th><th>  points_1</th></tr>
</thead>
<tbody>
<tr><td>12        </td><td>ashton town        </td><td>24      </td><td>9      </td><td>11    </td><td>29         </td><td>39             </t

In [34]:
eval_blury_string(outputs)

['1', '1', '1', '1', '1', '0', '0', '1', '0', '0', '0']

In [35]:
labels

['1', '1', '1', '1', '1', '0', '0', '1', '1', '0', '1']

In [39]:
result = [ ]
for i in range(len(table_loader.dataset)):
    sample = table_loader.normalize_table(table_loader.dataset[i])
    llm_chain = LLMChain(llm=model, prompt=stage_0_prompt, verbose=False)
    stage_0_batch_pred = llm_chain.batch([{"query": sample['query']}], return_only_outputs=True)[0]['text'].split(':')[-1]
    print(stage_0_batch_pred)
    sub_queries = stage_0_batch_pred.split(';')

    from langchain_community.callbacks import get_openai_callback
    Agent_history = ChatMessageHistory()
    agent_prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """Based on the history information, your task is to determine whether the information is enough to answer the user query.
                return 'A' if you think you need more information, else return 'B'. You should only use information during the conversation.""",
            ),
            MessagesPlaceholder(variable_name="chat_history"),
        ]
    )
    chain = LLMChain(llm=model, prompt=agent_prompt, verbose=True)
    with get_openai_callback() as cb:
        for sub_query in sub_queries:
            Agent_history.add_user_message(sub_query)
            choice = chain.invoke(
            {
                "chat_history": Agent_history.messages,
            }
            )['text']
            if 'A' in choice:
                Agent_history.add_ai_message(scene_A(sub_query, sample))
            else:
                res = scene_B()
                print(res)
                result.append(res)
                
    print(cb.total_tokens)
    

 What is the mintage of the 31997 release?; When was the 31997 release released?


> Entering new LLMChain chain...
Prompt after formatting:
System: Based on the history information, your task is to determine whether the information is enough to answer the user query.
                return 'A' if you think you need more information, else return 'B'. You should only use information during the conversation.
Human:  What is the mintage of the 31997 release?

> Finished chain.
year, theme, mintage
SELECT mintage
FROM DF
WHERE theme = 'hmcs bras dor' AND mintage = '31997';
Yes, the 31997 mintage was released in 2001.


> Entering new LLMChain chain...
Prompt after formatting:
System: Based on the history information, your task is to determine whether the information is enough to answer the user query.
                return 'A' if you think you need more information, else return 'B'. You should only use information during the conversation.
Human:  What is the mintage of the 31997 release?


OperationalError: (sqlite3.OperationalError) no such column: season
[SQL: SELECT record
FROM DF
WHERE season = 2005
AND team = 'Milwaukee Brewers';]
(Background on this error at: https://sqlalche.me/e/20/e3q8)